In [331]:
#!/usr/bin/env python
# coding: utf-8

# -------------------------------------------------------------------------------------------------------------
#   python3 requete-v1.py
#
#   Traitement d'une requête :
#       0. lecture fichier des index (inversé & documents)
#       1. lecture de la requête
#       2. extraction de ses tokens 
#       - lemmatisation
#       - normalisation 
#       3. recherche des documents correspondants dans l'index
#           par comptage des matchs
#       4. affichage des résultats
# 
#   remarque :  ne donne pas d'info sur les termes trouvés
# -------------------------------------------------------------------------------------------------------------

import os
import sys
import re
import pickle 
import json
#from fonctions_index import litTexteDuFichier, ecritTexteDansUnFichier, lemmatiseTexte, normaliseTokens, lemmatiseTermes, normaliseTokensRequete
import fonctions_index

PARSEUR = 'TALISMANE'
   
fiIndex = "./_index/indexInverse"
fiDocs = "./_index/indexDocuments"
doCorpus = "../corpus/miniCorpus/FR/"
corpus_sansBalise = "../corpus/sans_balises"

fiTxt = "./_log/tempo.txt" 

fiLog = "./_log/requete.log"
log = ""


#fonction pour lire le fichier indexDocument
def litIndexDocument(fichier):
    indexDocuments = dict()
    if os.path.isfile (fichier) :
        with open(fichier, 'rb') as FI :
            indexDocuments = json.load(FI)
    return indexDocuments

#fonction pour lire le fichier indexInverse
def litIndexInverse(fichier):
    indexInverse = dict ()
    if os.path.isfile (fichier) :
        with open(fichier, 'rb') as FI :
            indexInverse = json.load(FI)
    return indexInverse
 
#trier les termes saisis pour faire la reqête en trois parties
def trierTermesDeRequete(requete):
    termes_inclure = []
    termes_exclure = []
    termes_optionnel = []

    termes_avec_ponc = list(requete.split(" "))
    for terme in termes_avec_ponc:
        if terme.startswith("+"): 
            termes_inclure.append(terme[1:])
        elif terme.startswith("-"): 
            termes_exclure.append(terme[1:])
        else: termes_optionnel.append(terme)

    termes_totals = termes_inclure+termes_exclure+termes_optionnel
    return termes_totals, termes_inclure, termes_exclure, termes_optionnel


#recherche tous les termes dans l'index et stocker le résultat, retourne un résultat brut
def requeteDesTermes(termes, fichier_indexInverse):
    resultat_requete = dict ()
    for terme in termes:
        if terme in fichier_indexInverse:
            for liste in fichier_indexInverse[terme]:
                if resultat_requete.__contains__(liste[0]):
                    resultat_requete[liste[0]][terme] = liste[1]
                else:
                    valeur = {terme:liste[1]}
                    resultat_requete[liste[0]] = valeur
    return resultat_requete


#filtrer les résultats en fonctions des symboles "+" et "-"
def filtrerLeResultat(resultat_brut, termes_a_inclure, termes_a_exclure):
    def detecterLesTermes(document):
        termes = document[1]
        if all(terme in termes for terme in termes_a_inclure) == False:
            return False
        if any(terme in termes for terme in termes_a_exclure):
            return False
        return True
    nbMatch_filtre = dict(filter(detecterLesTermes, resultat_brut.items()))
    return nbMatch_filtre


def standardiseLesTermes(liste):
    liste = lemmatiseTermes(liste)
    liste = normaliseTokensRequete(liste)
    return liste

def lemmatiseTermes(liste):
    tokensLemme = []
    # execute TreeTagger
    texte = " ".join(liste)
    if detect(texte) == "fr":
        tagger = treetaggerwrapper.TreeTagger(TAGLANG="fr")
        tags = tagger.tag_text(texte)
    else:
        tagger = treetaggerwrapper.TreeTagger(TAGLANG="en")
        tags = tagger.tag_text(texte)
        lang = "EN"
    for ligne in tags:
        ligne = ligne.strip()
        defToken = ligne.split("\t")
        # filtre les tokens vides
        if len(defToken) >= 3:
            tokensLemme.append(defToken[2])
    # renvoie une liste de tokens (lemme et pos)
    return tokensLemme
# 6. calculer les scores avec TF-IDF



#----------------------------------main-------------------------------------------

#lire les fichiers de l'index de documents et l'index inversé
indexDocuments = litIndexDocument(fiDocs)
indexInverse = litIndexInverse(fiIndex)

#input la requête
print("votre requête ?")
requete = input()
print("votre requête est: ", requete)
log += "\nrequête: %s \n" % (requete)

#stocker les termes triés
termes_totals, termes_inclure, termes_exclure, termes_optionnel = trierTermesDeRequete(requete)
termes_totals_string = " ".join(termes_totals)
ecritTexteDansUnFichier (termes_totals_string, fiTxt) 

#lematiser et normaliser tous les termes de requête
tokens = lemmatiseTermes(termes_totals)
termes_totals_a_chercher = normaliseTokensRequete (tokens)
log += "\ntermes: %s \n" % (termes_totals_a_chercher)

#lamatiser et normaliser les termes iclures et les termes exculres
termes_inclure_final = standardiseLesTermes(termes_inclure)
termes_exclure_final = standardiseLesTermes(termes_exclure)
termes_optionnel_final = standardiseLesTermes(termes_optionnel)

#faire la reqête, chercher tous les documents qui comprennent tous les termes saisis, on obtient un résultat brut
nbMatch = requeteDesTermes(termes_totals_a_chercher, indexInverse)
log += "\nrésultat: %s \n" % (str(nbMatch))

#filtrer le résultat brut en fonction des symboles "+" et "-"
nbMatch_final = filtrerLeResultat(nbMatch, termes_inclure_final, termes_exclure_final)
log += "\nrésultats filtrés: %s\n" % str(nbMatch_final)

#affichage du résultat dans le terminal
print("\n{nombre} documents ont été trouvés!\n".format(nombre=len(nbMatch_final)))
print(nbMatch_final)


#extraire les sous-titres s'ils existent et les stocker dans "log"
log += "\nrésultats détaillées: \n"
for doc, nb in nbMatch_final.items():#sorted(nbMatch.items(), key=lambda item: item[0], reverse=True) :
    info_document = indexDocuments[str(doc)]
    nom_fichier = info_document[0]
    titre_complet = info_document[1]

    if "\n\n" in titre_complet:
        titre_principal = titre_complet[:titre_complet.index("\n\n")]
        sous_titre = titre_complet[titre_complet.index("\n\n")+2:]
        log += f"nom de fichier: {nom_fichier}\n titre de texte: {titre_principal}\n sous-titre: {sous_titre}\n termes trouvés dans ce fichier: {nb}\n\n"
    else:
        log += f"nom de fichier: {nom_fichier}\n titre de texte: {titre_complet}\n termes trouvés dans ce fichier: {nb}\n\n"



# sauvegarde du log
ecritTexteDansUnFichier (log, fiLog)

votre requête ?
Chômages citoyens +Économie +sociétés -salaire
votre requête est:  Chômages citoyens +Économie +sociétés -salaire

6 documents ont été trouvés!

{7: {'economie': 1, 'societe': 1}, 11: {'economie': 4, 'societe': 1}, 14: {'economie': 3, 'societe': 1, 'chomage': 3, 'citoyen': 1}, 15: {'economie': 1, 'societe': 4}, 22: {'economie': 2, 'societe': 1, 'chomage': 1, 'citoyen': 1}, 26: {'economie': 9, 'societe': 1}}


In [43]:
mots_list = "je -ne +sais -pas"
tokens = mots_list.split(" ")
tokens_sans_ponc=[]

print(tokens)
for i in tokens:
    if i[0] == "-" or i[0] == "+":
        tokens_sans_ponc.append(i[1:])
    else:
        tokens_sans_ponc.append(i)
cave = " ".join(tokens_sans_ponc)
cave

['je', '-ne', '+sais', '-pas']


'je ne sais pas'

In [38]:
aa = ""
aa.join(["s","o","c","i","a",'l'],split=" ")

TypeError: join() takes no keyword arguments

In [ ]:
#删除加减号的版本
def lemmatiseTokensRequete(fichier):
    tokensLemmePos = []
    fiParse = fichier + ".par"

    #supprimer le "+" et le "-" des termes de la requête
    requete_termes = litTexteDuFichier(fichier)
    termes_avec_ponc = requete_termes.split(" ")
    liste_termes_sans_ponc = []

    for terme in termes_avec_ponc:
        if terme[0] == "-" or terme[0] == "+":
            liste_termes_sans_ponc.append(terme[1:])
        else:
            liste_termes_sans_ponc.append(terme)
    termes_a_traiter = " ".join(liste_termes_sans_ponc)

    #écrire à nouveau les termes sans ponctuations dans le fichier "tempo.txt"
    ecritTexteDansUnFichier(termes_a_traiter, fichier)
    
    # execute TreeTagger
    if detect(requete_termes) == "fr":
        cmd = "tree-tagger-french"
    else:
        cmd = "tree-tagger-english"
    os.system("%s %s > %s" % (cmd, fichier, fiParse))

    # lit le csv produit par treeTagger
    with open(fiParse, "r") as FI:
        for ligne in FI:
            ligne = ligne.strip()
            defToken = ligne.split("\t")
            # filtre les tokens vides
            if len(defToken) >= 3:
                if defToken[2] != "<unknown>":
                    # ajouter les tokens lemmatisés dans la liste 
                    tokensLemmePos.append(defToken[2])
                else:
                    tokensLemmePos.append(defToken[0])
    # renvoie une liste de tokens (lemme)
    return tokensLemmePos

In [ ]:
# copie de la fonction de "fonctions_index.py"
def lemmatiseTokenRequete(fichier):
    tokensLemmePos = []
    fiParse = fichier + ".par"


    # execute TreeTagger
    cmd = "tree-tagger-french"
    os.system("%s %s > %s" % (cmd, fichier, fiParse))
    # lit le csv produit par treeTagger
    with open(fiParse, "r") as FI:
        for ligne in FI:
            ligne = ligne.strip()
            defToken = ligne.split("\t")
            # filtre les tokens vides
            if len(defToken) >= 3:
                if defToken[2] != "<unknown>":
                    # ajouter les tokens lemmatisés dans la liste 
                    tokensLemmePos.append(defToken[2])
                else:
                    tokensLemmePos.append(defToken[0])
    # renvoie une liste de tokens (lemme)
    return tokensLemmePos

In [266]:
resultats = {14: {'chomage': 3, 'citoyen': 1, 'economie': 3, 'societe': 1}, 22: {'chomage': 1, 'citoyen': 1, 'economie': 2, 'societe': 1}, 0: {'citoyen': 1, 'economie': 1}, 5: {'citoyen': 2, 'societe': 1}, 10: {'citoyen': 3, 'societe': 5}, 4: {'economie': 4, 'societe': 3, 'salaire': 3}, 7: {'economie': 1, 'societe': 1}, 11: {'economie': 4, 'societe': 1}, 15: {'economie': 1, 'societe': 4}, 26: {'economie': 9, 'societe': 1}} 

In [267]:
resultats

{14: {'chomage': 3, 'citoyen': 1, 'economie': 3, 'societe': 1},
 22: {'chomage': 1, 'citoyen': 1, 'economie': 2, 'societe': 1},
 0: {'citoyen': 1, 'economie': 1},
 5: {'citoyen': 2, 'societe': 1},
 10: {'citoyen': 3, 'societe': 5},
 4: {'economie': 4, 'societe': 3, 'salaire': 3},
 7: {'economie': 1, 'societe': 1},
 11: {'economie': 4, 'societe': 1},
 15: {'economie': 1, 'societe': 4},
 26: {'economie': 9, 'societe': 1}}

In [268]:
mots_list = []
for i in resultats:
    mots_list.append(i)


In [269]:
mots_list

[14, 22, 0, 5, 10, 4, 7, 11, 15, 26]

In [151]:
import copy
resultats_finals = copy.deepcopy(resultats)


In [152]:
resultats_finals

{14: {'chomage': 3, 'citoyen': 1, 'economie': 3, 'societe': 1},
 22: {'chomage': 1, 'citoyen': 1, 'economie': 2, 'societe': 1},
 0: {'citoyen': 1, 'economie': 1},
 5: {'citoyen': 2, 'societe': 1},
 10: {'citoyen': 3, 'societe': 5},
 4: {'economie': 4, 'societe': 3, 'salaire': 3},
 7: {'economie': 1, 'societe': 1},
 11: {'economie': 4, 'societe': 1},
 15: {'economie': 1, 'societe': 4},
 26: {'economie': 9, 'societe': 1}}

In [153]:
requete = ["Chômages","citoyens","+Économie","+sociétés","-salaire"]

In [154]:
requete_finale = ["chomage","citoyen","+economie","+societe","-salaire"]#7,11,15,26

In [163]:
rrr = {}
for document in resultats_finals:
    for terme in requete_finale:
        if terme.startswith("-") and terme[1:] in resultats_finals[document]:
            resultats_finals.pop(document)
        else:
            pass

KeyError: 10

In [197]:
ag = list(resultats_finals.items())
mots_a_traiter = ag[0][1]

In [198]:
mots_a_traiter

{'economie': 4, 'societe': 3, 'salaire': 3}

In [201]:
all(term in mots_a_traiter for term in requete_finale)

False

In [207]:
a = (term in mots_a_traiter for term in requete_finale)

In [209]:
list(a)

[False, False, False, False, False]

In [213]:
a = list(term in mots_a_traiter for term in requete_finale)
a

[False, False, False, False, False]

In [214]:
for terme in requete_finale:
    for document in resultats_finals:
        if (terme[0] == "+") and (terme[1:] in resultats_finals[document]):
            if document in rrr:
                pass
            else:
                rrr[document] = resultats_finals.get(document)
        

In [219]:
inputt = input()
aa = list(inputt.split(" "))
print(aa)

dfs +dsdfd -sfe +dd
['dfs', '+dsdfd', '-sfe', '+dd']


In [240]:
import treetaggerwrapper

In [241]:
tokensLemmePos = []
    texte = extraitTexteDuFichier(fichier, nom_fichier)
    if detect(texte) == "fr":
        tagger = treetaggerwrapper.TreeTagger(TAGLANG="fr")
        tags = tagger.tag_text(texte)
        lang = "FR"
    else:
        tagger = treetaggerwrapper.TreeTagger(TAGLANG="en")
        tags = tagger.tag_text(texte)
        lang = "EN"
    for ligne in tags:
        ligne = ligne.strip()
        defToken = ligne.split("\t")
        # filtre les tokens vides
        if len(defToken) >= 3:
            # reduit aux lemmes et pos
            token = [defToken[2], defToken[1]]
            tokensLemmePos.append(token)
    # renvoie une liste de tokens (lemme et pos)
    return tokensLemmePos, lang

IndentationError: unexpected indent (<ipython-input-241-ae39d607adce>, line 2)

In [246]:
termes_test = ["chômages","salaires","citoyenne"]

In [248]:
tagger = treetaggerwrapper.TreeTagger(TAGLANG="fr")
aa = tagger.tag_text(termes_test)

In [249]:
aa

['chômages\tNOM\tchômage', 'salaires\tNOM\tsalaire', 'citoyenne\tNOM\tcitoyen']

In [256]:
for i in aa:
    i = i.strip()
    token = i.split("\t")
    

In [259]:
text = " ".join(termes_test)
text

'chômages salaires citoyenne'

In [254]:
from langdetect import detect

In [260]:
detect(text)

'fr'

In [261]:
tagger = treetaggerwrapper.TreeTagger(TAGLANG="fr")
aa = tagger.tag_text(text)

In [262]:
aa

['chômages\tNOM\tchômage', 'salaires\tNOM\tsalaire', 'citoyenne\tNOM\tcitoyen']

In [263]:
def lemmatiseTermes(liste):
    tokensLemme = []
    # execute TreeTagger
    texte = " ".join(liste)
    if detect(texte) == "fr":
        tagger = treetaggerwrapper.TreeTagger(TAGLANG="fr")
        tags = tagger.tag_text(texte)
    else:
        tagger = treetaggerwrapper.TreeTagger(TAGLANG="en")
        tags = tagger.tag_text(texte)
        lang = "EN"
    for ligne in tags:
        ligne = ligne.strip()
        defToken = ligne.split("\t")
        # filtre les tokens vides
        if len(defToken) >= 3:
            # reduit aux lemmes et pos
            token = [defToken[2]]
            tokensLemme.append(token)
    # renvoie une liste de tokens (lemme et pos)
    return tokensLemme

In [264]:
lemmatiseTermes(termes_test)

[['chômage'], ['salaire'], ['citoyen']]

In [ ]:

def lemmatiseTokensRequete(fichier):
    tokensLemmePos = []
    fiParse = fichier + ".par"
    
    # execute TreeTagger
    termes_requete = litTexteDuFichier(fichier)
    if detect(termes_requete) == "fr":
        cmd = "tree-tagger-french"
    else:
        cmd = "tree-tagger-english"
    os.system("%s %s > %s" % (cmd, fichier, fiParse))

    # lit le csv produit par treeTagger
    with open(fiParse, "r") as FI:
        for ligne in FI:
            ligne = ligne.strip()
            defToken = ligne.split("\t")
            # filtre les tokens vides
            if len(defToken) >= 3:
                if defToken[2] != "<unknown>":
                    # ajouter les tokens lemmatisés dans la liste 
                    tokensLemmePos.append(defToken[2])
                else:
                    tokensLemmePos.append(defToken[0])
    # renvoie une liste de tokens (lemme)
    return tokensLemmePos

In [270]:
indexInverse

{'-dominated': [[6, 1]],
 '-qui': [[8, 1]],
 '000-strong': [[13, 1]],
 '1.3bn': [[25, 1]],
 '1.6bn': [[25, 1]],
 '1.9bn': [[25, 1]],
 '100bn': [[25, 1]],
 '10bn': [[19, 1], [25, 1]],
 '10m': [[28, 1]],
 '11-hour': [[21, 1]],
 '18.522bn': [[28, 1]],
 '1920s': [[24, 1]],
 '1930s': [[24, 1]],
 '1940s': [[24, 2]],
 '1950s': [[24, 1]],
 '1960s': [[3, 1], [6, 1], [18, 1]],
 '1970s': [[3, 1], [6, 1], [18, 1], [20, 1], [24, 1]],
 '1980s': [[6, 6], [18, 1], [29, 1]],
 '1990s': [[2, 1], [6, 2], [13, 1], [21, 1], [24, 1]],
 '19th': [[25, 1]],
 '1bn': [[3, 1]],
 '1er': [[5, 2], [22, 1]],
 '2.2m': [[19, 1]],
 '2.8bn': [[28, 1]],
 '2.955bn': [[28, 1]],
 '200m': [[25, 1]],
 '22bn': [[28, 1]],
 '24bn': [[19, 1]],
 '2bn': [[19, 1]],
 '2m': [[24, 1]],
 '3.28m': [[19, 1]],
 '30-foot-high': [[24, 1]],
 '37e': [[27, 1]],
 '3m': [[1, 1], [19, 1]],
 '4.42m': [[19, 1]],
 '400bn': [[6, 1]],
 '4bn': [[28, 1]],
 '4e': [[27, 1]],
 '4m': [[19, 1]],
 '4th': [[1, 1]],
 '4x4': [[8, 1]],
 '50m': [[1, 1]],
 '51st': [[2

In [271]:
tousLesTermes = []
for terme in indexInverse:
    tousLesTermes.append(terme)

In [272]:
tousLesTermes

['-dominated',
 '-qui',
 '000-strong',
 '1.3bn',
 '1.6bn',
 '1.9bn',
 '100bn',
 '10bn',
 '10m',
 '11-hour',
 '18.522bn',
 '1920s',
 '1930s',
 '1940s',
 '1950s',
 '1960s',
 '1970s',
 '1980s',
 '1990s',
 '19th',
 '1bn',
 '1er',
 '2.2m',
 '2.8bn',
 '2.955bn',
 '200m',
 '22bn',
 '24bn',
 '2bn',
 '2m',
 '3.28m',
 '30-foot-high',
 '37e',
 '3m',
 '4.42m',
 '400bn',
 '4bn',
 '4e',
 '4m',
 '4th',
 '4x4',
 '50m',
 '51st',
 '55e',
 '55th',
 '5e',
 '5th',
 '600bn',
 '600m',
 '6e',
 '6th',
 '70bn',
 '7bn-',
 '7e',
 '7th',
 '800m',
 '80bn',
 '81bn',
 '@card@',
 '@ord@',
 'a-t-il',
 'ababa',
 'abandon',
 'abc',
 'abces',
 'abdallah',
 'abdel',
 'abdul',
 'abdullah',
 'abeba',
 'abet',
 'ability',
 'able',
 'abm',
 'aboi',
 'abolish',
 'abolition',
 'abord',
 'abou',
 'abrahams',
 'abri',
 'absence',
 'absolute',
 'absolution',
 'absorb',
 'abu',
 'abuja',
 'abuse',
 'abusif',
 'academic',
 'accede',
 'accept',
 'acceptance',
 'acces',
 'access',
 'accession',
 'accommodate',
 'accommodation',
 'accom

In [273]:
import numpy as np

In [286]:
ee = np.zeros([len(indexDocuments),len(indexInverse)])

In [334]:
ee

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [335]:
nbMatch_final

{7: {'economie': 1, 'societe': 1},
 11: {'economie': 4, 'societe': 1},
 14: {'economie': 3, 'societe': 1, 'chomage': 3, 'citoyen': 1},
 15: {'economie': 1, 'societe': 4},
 22: {'economie': 2, 'societe': 1, 'chomage': 1, 'citoyen': 1},
 26: {'economie': 9, 'societe': 1}}

In [347]:
termes_totals

['Économie', 'sociétés', 'salaire', 'Chômages', 'citoyens']

In [348]:
essai = np.zeros([len(nbMatch_final),len(termes_totals)])

In [346]:
essai

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [393]:
essai_docs = {"0":"aaaa","1":"bbbb","2":"CCCC","3":"dddd","4":"EEEE"}
essai_termes = {"toujours":[[0,2],[2,3],[4,1]], "oui":[[0,3],[1,1],[2,8]], "pour":[[3,1]], "economie":[[1,1],[2,2],[3,1]]}


In [455]:
rr = ["oui","pour","toujours"]

In [ ]:
idf = math.log(TFIDF.shape[0] / (np.count_nonzero(matrice[:,j]))) 

In [394]:

def creerMatrice(document_index, index_inverse):
    #créer une liste de tous les termes 
    listeDesTermes = []
    for terme in index_inverse:
        listeDesTermes.append(terme)

    #créer une matrice
    matrice = np.zeros([len(document_index),len(index_inverse)])

    #ajouter les occurrences dans la matrice
    for doc in document_index:
        for terme in index_inverse:
            for couple in index_inverse.get(terme):
                if str(couple[0]) == doc:
                    matrice[int(doc), listeDesTermes.index(terme)] = couple[1]
    return matrice


In [399]:
matrice_brut = creerMatrice(essai_docs,essai_termes)
print(matrice_brut)

[[2. 3. 0. 0.]
 [0. 1. 0. 1.]
 [3. 8. 0. 2.]
 [0. 0. 1. 1.]
 [1. 0. 0. 0.]]


In [405]:
matrice_brut.shape[1]

4

In [428]:
matrice_brut[3]

array([0., 0., 1., 1.])

In [396]:
import math

In [411]:
def TFIDF(matrice):
    matrice_TFIDF = np.zeros(matrice.shape)
    for i in range (TFIDF.shape[0]):
        for j in range (TFIDF.shape[1]):
          tf = matrice[i,j]
          idf = math.log(TFIDF.shape[0] /(np.count_nonzero(matrice[:,j])))  
          TFIDF[i,j] = tf * idf 
    return matrice_TFIDF

In [412]:
TFIDF(matrice_brut)

array([[1.02165125, 1.53247687, 0.        , 0.        ],
       [0.        , 0.51082562, 0.        , 0.51082562],
       [1.53247687, 4.08660499, 0.        , 1.02165125],
       [0.        , 0.        , 1.60943791, 0.51082562],
       [0.51082562, 0.        , 0.        , 0.        ]])

TypeError: 'function' object is not subscriptable

In [413]:
aaa = creerMatrice(indexDocuments,indexInverse)

In [414]:
aaa

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 1.]])

In [415]:
TFIDF(aaa)

array([[0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 3.4657359,
        0.       ],
       [0.       , 0.       , 0.       , ..., 3.4657359, 0.       ,
        3.4657359]])

In [495]:
termes_positifs = termes_optionnel_final + termes_inclure_final
print(termes_positifs)
nbMatch_final

['chomage', 'citoyen', 'economie', 'societe']


{7: {'economie': 1, 'societe': 1},
 11: {'economie': 4, 'societe': 1},
 14: {'economie': 3, 'societe': 1, 'chomage': 3, 'citoyen': 1},
 15: {'economie': 1, 'societe': 4},
 22: {'economie': 2, 'societe': 1, 'chomage': 1, 'citoyen': 1},
 26: {'economie': 9, 'societe': 1}}

In [ ]:


for id_doc, occus in nbMatch_final.items():
    v = np.array([occus.get(word, 0) for word in termes_positifs])
    
    
    """
    vecteur = self.pond( v )
    docs_similarity[id] = self.score_func( vecteur, vec_ref )
    """

In [492]:
def vec_termesRequete(listeDesTermes, doc_idf):
    tf = 1 / len(listeDesTermes)
    tfidf_liste = []
    for terme in listeDesTermes:
        if terme in doc_idf:
            tf_idf = tf * doc_idf[terme]
            tfidf_liste.append(tf_idf)
    return tfidf_liste

In [493]:
vec_termesRequete(termes_positifs,liste_idf_essai)

[0.34657359027997264,
 0.3171278313658768,
 0.6931471805599453,
 0.46407449759140657]

In [425]:
from numpy import dot
from numpy.linalg import norm
score_consinu = lambda v1, v2 : dot(v1, v2)/(norm(v1)*norm(v2))

In [ ]:
idf = math.log(TFIDF.shape[0] /(np.count_nonzero(matrice[:,j]))) 

In [470]:
def getIDF(doc_indexInverse, doc_indexDocuments):
    dicIDF = {}
    for terme in doc_indexInverse:
        idf = math.log(len(doc_indexDocuments) / len(doc_indexInverse[terme]))
        dicIDF[terme] = idf
    return dicIDF

In [473]:
liste_idf_essai = getIDF(indexInverse, indexDocuments)

In [474]:
liste_idf_essai

{'-dominated': 3.4657359027997265,
 '-qui': 3.4657359027997265,
 '000-strong': 3.4657359027997265,
 '1.3bn': 3.4657359027997265,
 '1.6bn': 3.4657359027997265,
 '1.9bn': 3.4657359027997265,
 '100bn': 3.4657359027997265,
 '10bn': 2.772588722239781,
 '10m': 3.4657359027997265,
 '11-hour': 3.4657359027997265,
 '18.522bn': 3.4657359027997265,
 '1920s': 3.4657359027997265,
 '1930s': 3.4657359027997265,
 '1940s': 3.4657359027997265,
 '1950s': 3.4657359027997265,
 '1960s': 2.367123614131617,
 '1970s': 1.8562979903656263,
 '1980s': 2.367123614131617,
 '1990s': 1.8562979903656263,
 '19th': 3.4657359027997265,
 '1bn': 3.4657359027997265,
 '1er': 2.772588722239781,
 '2.2m': 3.4657359027997265,
 '2.8bn': 3.4657359027997265,
 '2.955bn': 3.4657359027997265,
 '200m': 3.4657359027997265,
 '22bn': 3.4657359027997265,
 '24bn': 3.4657359027997265,
 '2bn': 3.4657359027997265,
 '2m': 3.4657359027997265,
 '3.28m': 3.4657359027997265,
 '30-foot-high': 3.4657359027997265,
 '37e': 3.4657359027997265,
 '3m': 2.7

In [504]:
#retourner le idf de tous les termes dans indexInverse
def getIDF(doc_indexInverse, doc_indexDocuments):
    dicIDF = {}
    for terme in doc_indexInverse:
        idf = math.log(len(doc_indexDocuments) / len(doc_indexInverse[terme]))
        dicIDF[terme] = idf
    return dicIDF


#retourner le vecteur de la requete(tf-idf), avec les termes à inclure et les termes optionnels
def vec_termesRequete(listeDesTermes, doc_idf):
    tf = 1 / len(listeDesTermes)
    tfidf_liste = []
    for terme in listeDesTermes:
        if terme in doc_idf:
            tf_idf = tf * doc_idf[terme]
            tfidf_liste.append(tf_idf)
    return tfidf_liste



In [505]:


vec_termesRequete(termes_positifs, liste_idf_essai)

[0.6931471805599453,
 0.46407449759140657,
 0.34657359027997264,
 0.3171278313658768]

In [513]:
o = {2:4, "d":33, "r":[[3,5,0],"score"], }

In [514]:
o

{2: 4, 'd': 33, 'r': [[3, 5, 0], 'score']}